# How to connect to OpenAI service via PromptSail and Langchain SDK

First import the OpenAI Python SDK and load your API key from the environment.


In [1]:
import os
from dotenv import load_dotenv, dotenv_values
from pprint import pprint

config = dotenv_values(".env")

openai_key = config["OPENAI_API_KEY"]
openai_org_id = config["OPENAI_ORG_ID"]
print(
    f"OpenAI api key={openai_key[0:3]}...{openai_key[-3:]}"
)

OpenAI api key=sk-...Uzy


Test the direct connection to OpenAI and Langchain SDK.


In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.schema import HumanMessage, SystemMessage


messages = [
    SystemMessage(
        content="You are a helpful assistant that help rewirte an jira ticket."
    ),
    HumanMessage(
        content="Give meaningful title to this bug, RuntimeError: CUDA out of memory. Tried to allocate X MiB (GPU X; X GiB total capacity; X GiB already allocated; X MiB free; X cached)"
    ),
]


In [4]:

chat = ChatOpenAI(
    temperature=0,
    openai_api_key=openai_key,
    openai_organization=openai_org_id,
    model="gpt-3.5-turbo",
)

In [5]:
chat(messages)

AIMessage(content='Bug: Out of Memory Error when Allocating CUDA Memory (GPU X)')

## Create a request to the OpenAI via PromptSail proxy

Run the docker and go to PromptSail UI http://promptsail.local/


At the application start we created two test projects with OpenAI API mappings. We will use project1 for this example.

* [project1](http://promptsail.local/ui/project/project1) -> https://api.openai.com/v1

You can create your own project if you want.



In [11]:
ps_api_base = "http://promptsail.local:8000/project1/"
chat = ChatOpenAI(
    temperature=0,
    openai_api_key=openai_key,
    openai_organization=openai_org_id,
    model="gpt-3.5-turbo",
    base_url=ps_api_base,
)


In [6]:
chat(messages)

AIMessage(content='Bug: Out of Memory Error when Allocating CUDA Memory (GPU X)')